In [3]:
from entities import *
from constants import TEAMS_DATA, STAT_EFFECTS, POSSESSIONS_STD_DEV
from simulations import *

import pickle
import numpy as np
import pandas as pd

In [4]:
with open(TEAMS_DATA, 'rb') as f:
    teams = pickle.load(f)

with open(STAT_EFFECTS, 'rb') as f:
    stats = pickle.load(f)

In [5]:
az = teams['Arizona']
af = teams['Air Force']

In [15]:
simulate_game(az, af)

{'Win': 'Arizona',
 'Win Score': 99.05018695842647,
 'Lose': 'Air Force',
 'Lose Score': 59.386494629401156,
 'Game Log': [{"Anthony Dell'Orso": [15.716869037334591,
    0.6513497744454098,
    {'Pts': 14.09332638843641,
     'BLK': 0.4315296361932824,
     'STL': 1.7980401508053434,
     'TOV': 2.5891778171596944,
     'ORB': 0.4315296361932824,
     'DRB': 2.5891778171596944,
     'PF': 2.229569786998626}],
   'Dwayne Aristode': [17.007354089517346,
    -0.6427674483776944,
    {'Pts': 14.193872793669446,
     'BLK': 0.0,
     'STL': 0.57537284825771,
     'TOV': 0.8630592723865648,
     'ORB': 1.7980401508053434,
     'DRB': 3.2364722714496184,
     'PF': 1.7980401508053434}],
   'Ivan Kharchenkov': [16.251299814554947,
    -2.3279775493549653,
    {'Pts': 11.216174460723732,
     'BLK': 0.6472944542899236,
     'STL': 2.5172562111274805,
     'TOV': 2.085726574934198,
     'ORB': 1.5103537266764886,
     'DRB': 5.034512422254961,
     'PF': 2.5172562111274805}],
   'Jaden Bradley':